<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/446_PDO_StatisticalAssessment_Utils.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""Statistical Assessment Utilities for Proposal & Document Orchestrator

These utilities add statistical rigor to KPI and ROI analysis using toolshed.statistics.
Following CEO Trust patterns: statistical significance, confidence intervals, trend analysis.

Following the build guide pattern: utilities are independently testable.
"""

from typing import Dict, Any, List, Optional
from toolshed.statistics import (
    test_kpi_significance,
    test_roi_significance,
    test_trend_significance,
    calculate_kpi_confidence_interval,
    calculate_roi_confidence_interval,
    assess_kpi_with_significance,
    assess_roi_with_significance
)


def assess_kpi_statistical_significance(
    kpi_name: str,
    current_value: float,
    historical_values: Optional[List[float]] = None,
    target_value: Optional[float] = None,
    confidence_level: float = 0.95
) -> Dict[str, Any]:
    """
    Assess KPI with statistical significance testing.

    Args:
        kpi_name: Name of the KPI
        current_value: Current KPI value
        historical_values: Optional list of historical values
        target_value: Optional target value
        confidence_level: Confidence level (default 0.95)

    Returns:
        Dictionary with statistical assessment:
        {
            "kpi_name": str,
            "current_value": float,
            "statistical_test": {...},
            "confidence_interval": {...},
            "target_comparison": {...},
            "assessment": str
        }
    """
    result = {
        "kpi_name": kpi_name,
        "current_value": current_value,
        "has_historical_data": historical_values is not None and len(historical_values) >= 2
    }

    # Statistical significance test (if historical data available)
    if historical_values and len(historical_values) >= 2:
        statistical_test = test_kpi_significance(
            current_value=current_value,
            historical_values=historical_values,
            target_value=target_value,
            confidence_level=confidence_level
        )
        result["statistical_test"] = statistical_test

        # Confidence interval
        try:
            ci = calculate_kpi_confidence_interval(
                kpi_value=current_value,
                sample_size=len(historical_values),
                confidence_level=confidence_level,
                is_proportion=(0 <= current_value <= 1)
            )
            result["confidence_interval"] = ci
        except Exception:
            result["confidence_interval"] = None
    else:
        result["statistical_test"] = {
            "error": "Insufficient historical data for statistical testing",
            "is_significant": None
        }
        result["confidence_interval"] = None

    # Comprehensive assessment (if historical data available)
    if historical_values and len(historical_values) >= 2:
        try:
            assessment = assess_kpi_with_significance(
                current_value=current_value,
                historical_values=historical_values,
                target_value=target_value,
                confidence_level=confidence_level
            )
            result["comprehensive_assessment"] = assessment
            result["assessment"] = assessment.get("executive_summary", "Statistical assessment completed")
        except Exception as e:
            result["assessment"] = f"Assessment completed (limited: {str(e)})"
    else:
        result["assessment"] = "No historical data available for statistical assessment"

    return result


def assess_roi_statistical_significance(
    roi_estimate: float,
    cost: float,
    historical_roi: Optional[List[float]] = None,
    confidence_level: float = 0.95
) -> Dict[str, Any]:
    """
    Assess ROI with statistical significance testing.

    Args:
        roi_estimate: Current ROI estimate
        cost: Current cost
        historical_roi: Optional list of historical ROI values
        confidence_level: Confidence level (default 0.95)

    Returns:
        Dictionary with ROI statistical assessment:
        {
            "roi_estimate": float,
            "cost": float,
            "statistical_test": {...},
            "confidence_interval": {...},
            "assessment": str
        }
    """
    result = {
        "roi_estimate": roi_estimate,
        "cost": cost,
        "roi_ratio": roi_estimate / cost if cost > 0 else float('inf'),
        "has_historical_data": historical_roi is not None and len(historical_roi) >= 2
    }

    # Statistical significance test (if historical data available)
    if historical_roi and len(historical_roi) >= 2:
        statistical_test = test_roi_significance(
            roi_estimate=roi_estimate,
            cost=cost,
            historical_roi=historical_roi,
            confidence_level=confidence_level
        )
        result["statistical_test"] = statistical_test

        # Confidence interval
        try:
            ci = calculate_roi_confidence_interval(
                roi_estimate=roi_estimate,
                cost=cost,
                historical_roi=historical_roi,
                confidence_level=confidence_level
            )
            result["confidence_interval"] = ci
        except Exception:
            result["confidence_interval"] = None
    else:
        result["statistical_test"] = {
            "error": "Insufficient historical data for statistical testing",
            "is_significantly_positive": None
        }
        result["confidence_interval"] = None

    # Comprehensive assessment (if historical data available)
    if historical_roi and len(historical_roi) >= 2:
        try:
            assessment = assess_roi_with_significance(
                roi_estimate=roi_estimate,
                cost=cost,
                historical_roi=historical_roi,
                confidence_level=confidence_level
            )
            result["comprehensive_assessment"] = assessment
            result["assessment"] = assessment.get("executive_summary", "ROI statistical assessment completed")
        except Exception as e:
            result["assessment"] = f"Assessment completed (limited: {str(e)})"
    else:
        result["assessment"] = "No historical data available for statistical assessment"

    return result


def analyze_kpi_trends(
    kpi_name: str,
    values: List[float],
    confidence_level: float = 0.95
) -> Dict[str, Any]:
    """
    Analyze trends in KPI values over time.

    Args:
        kpi_name: Name of the KPI
        values: List of KPI values over time (chronologically ordered)
        confidence_level: Confidence level (default 0.95)

    Returns:
        Dictionary with trend analysis:
        {
            "kpi_name": str,
            "trend_analysis": {...},
            "trend_direction": str,
            "is_significant": bool
        }
    """
    if len(values) < 3:
        return {
            "kpi_name": kpi_name,
            "trend_analysis": {
                "error": "Insufficient data for trend analysis (need at least 3 observations)"
            },
            "trend_direction": "insufficient_data",
            "is_significant": False
        }

    # Test trend significance
    trend_analysis = test_trend_significance(
        values=values,
        confidence_level=confidence_level
    )

    return {
        "kpi_name": kpi_name,
        "trend_analysis": trend_analysis,
        "trend_direction": trend_analysis.get("trend_direction", "unknown"),
        "is_significant": trend_analysis.get("is_significant", False)
    }


def assess_all_kpis_statistically(
    operational_kpis: Dict[str, Any],
    effectiveness_kpis: Dict[str, Any],
    business_kpis: Dict[str, Any],
    historical_data: Optional[Dict[str, List[float]]] = None,
    config: Any = None
) -> Dict[str, Any]:
    """
    Assess all KPIs with statistical significance testing.

    Args:
        operational_kpis: Operational KPI metrics
        effectiveness_kpis: Effectiveness KPI metrics
        business_kpis: Business KPI metrics
        historical_data: Optional dictionary mapping KPI names to historical values
        config: Agent configuration (for confidence_level and targets)

    Returns:
        Dictionary with statistical assessments for all KPIs:
        {
            "operational_kpis": {...},
            "effectiveness_kpis": {...},
            "business_kpis": {...},
            "summary": {...}
        }
    """
    confidence_level = getattr(config, "confidence_level", 0.95) if config else 0.95

    assessments = {
        "operational_kpis": {},
        "effectiveness_kpis": {},
        "business_kpis": {},
        "summary": {
            "total_kpis_assessed": 0,
            "kpis_with_historical_data": 0,
            "statistically_significant_improvements": 0,
            "statistically_significant_declines": 0
        }
    }

    # Assess operational KPIs
    for kpi_name, kpi_value in operational_kpis.items():
        historical_values = historical_data.get(kpi_name) if historical_data else None
        target_value = getattr(config, f"target_{kpi_name}", None) if config else None

        assessment = assess_kpi_statistical_significance(
            kpi_name=kpi_name,
            current_value=kpi_value,
            historical_values=historical_values,
            target_value=target_value,
            confidence_level=confidence_level
        )

        assessments["operational_kpis"][kpi_name] = assessment
        assessments["summary"]["total_kpis_assessed"] += 1

        if assessment.get("has_historical_data"):
            assessments["summary"]["kpis_with_historical_data"] += 1
            if assessment.get("statistical_test", {}).get("is_significant"):
                if assessment["statistical_test"].get("difference", 0) > 0:
                    assessments["summary"]["statistically_significant_improvements"] += 1
                else:
                    assessments["summary"]["statistically_significant_declines"] += 1

    # Assess effectiveness KPIs
    for kpi_name, kpi_value in effectiveness_kpis.items():
        historical_values = historical_data.get(kpi_name) if historical_data else None
        target_value = getattr(config, f"target_{kpi_name}", None) if config else None

        assessment = assess_kpi_statistical_significance(
            kpi_name=kpi_name,
            current_value=kpi_value,
            historical_values=historical_values,
            target_value=target_value,
            confidence_level=confidence_level
        )

        assessments["effectiveness_kpis"][kpi_name] = assessment
        assessments["summary"]["total_kpis_assessed"] += 1

        if assessment.get("has_historical_data"):
            assessments["summary"]["kpis_with_historical_data"] += 1
            if assessment.get("statistical_test", {}).get("is_significant"):
                if assessment["statistical_test"].get("difference", 0) > 0:
                    assessments["summary"]["statistically_significant_improvements"] += 1
                else:
                    assessments["summary"]["statistically_significant_declines"] += 1

    # Assess business KPIs
    for kpi_name, kpi_value in business_kpis.items():
        historical_values = historical_data.get(kpi_name) if historical_data else None
        target_value = getattr(config, f"target_{kpi_name}", None) if config else None

        assessment = assess_kpi_statistical_significance(
            kpi_name=kpi_name,
            current_value=kpi_value,
            historical_values=historical_values,
            target_value=target_value,
            confidence_level=confidence_level
        )

        assessments["business_kpis"][kpi_name] = assessment
        assessments["summary"]["total_kpis_assessed"] += 1

        if assessment.get("has_historical_data"):
            assessments["summary"]["kpis_with_historical_data"] += 1
            if assessment.get("statistical_test", {}).get("is_significant"):
                if assessment["statistical_test"].get("difference", 0) > 0:
                    assessments["summary"]["statistically_significant_improvements"] += 1
                else:
                    assessments["summary"]["statistically_significant_declines"] += 1

    return assessments


def calculate_statistical_assessments(
    operational_kpis: Dict[str, Any],
    effectiveness_kpis: Dict[str, Any],
    business_kpis: Dict[str, Any],
    total_cost_usd: float,
    net_roi_usd: float,
    historical_data: Optional[Dict[str, Any]] = None,
    config: Any = None
) -> Dict[str, Any]:
    """
    Calculate complete statistical assessments for KPIs and ROI.

    Args:
        operational_kpis: Operational KPI metrics
        effectiveness_kpis: Effectiveness KPI metrics
        business_kpis: Business KPI metrics
        total_cost_usd: Total cost
        net_roi_usd: Net ROI
        historical_data: Optional historical data dictionary
        config: Agent configuration

    Returns:
        Complete statistical assessments:
        {
            "kpi_assessments": {...},
            "roi_assessment": {...},
            "confidence_intervals": {...},
            "trends": {...}
        }
    """
    confidence_level = getattr(config, "confidence_level", 0.95) if config else 0.95

    # Extract historical KPI data
    historical_kpi_data = None
    historical_roi_data = None

    if historical_data:
        historical_kpi_data = historical_data.get("kpis", {})
        historical_roi_data = historical_data.get("roi", [])

    # Assess all KPIs
    kpi_assessments = assess_all_kpis_statistically(
        operational_kpis=operational_kpis,
        effectiveness_kpis=effectiveness_kpis,
        business_kpis=business_kpis,
        historical_data=historical_kpi_data,
        config=config
    )

    # Assess ROI
    roi_assessment = assess_roi_statistical_significance(
        roi_estimate=net_roi_usd,
        cost=total_cost_usd,
        historical_roi=historical_roi_data,
        confidence_level=confidence_level
    )

    # Extract confidence intervals
    confidence_intervals = {}

    # KPI confidence intervals
    for category in ["operational_kpis", "effectiveness_kpis", "business_kpis"]:
        for kpi_name, assessment in kpi_assessments[category].items():
            ci = assessment.get("confidence_interval")
            if ci:
                confidence_intervals[kpi_name] = ci

    # ROI confidence interval
    if roi_assessment.get("confidence_interval"):
        confidence_intervals["roi"] = roi_assessment["confidence_interval"]

    # Trend analysis (if we have time series data)
    trends = {}
    # For MVP, we'll skip trend analysis as it requires time series data
    # This can be added later when historical data is available

    return {
        "kpi_assessments": kpi_assessments,
        "roi_assessment": roi_assessment,
        "confidence_intervals": confidence_intervals,
        "trends": trends
    }




# Executive-Level Verdict

✅ **This utilities module is production-grade MVP quality**
✅ It aligns perfectly with your *CEO Trust* and *proof-before-LLM* philosophy
✅ It is safe, defensive, and independently testable
⚠️ A few **small refinements** will future-proof it and simplify orchestration

Nothing here is fundamentally wrong. This is *refinement*, not correction.

---

# What You Did Exceptionally Well (Do Not Change)

## 1. Separation of Concerns Is Excellent

You nailed this pattern:

* KPI calculation → **measurement**
* Statistical utilities → **validation**
* Report generation → **communication**

This module:

* Does **no I/O**
* Does **no state mutation**
* Does **no formatting**
* Returns structured, auditable data

That’s exactly how this should look.

---

## 2. Defensive Handling of Missing Data = Enterprise-Safe

This is *very* well done:

```python
if historical_values and len(historical_values) >= 2:
```

and:

```python
"has_historical_data": historical_values is not None and len(historical_values) >= 2
```

Why this matters:

* No silent failures
* No fake “confidence”
* Clear signaling when rigor is unavailable

Executives trust systems that say:

> “We don’t have enough data yet.”

This module does that correctly.

---

## 3. You Used Toolshed Correctly (Not Naively)

You did **not**:

* Reimplement statistics
* Overwrap toolshed
* Hide results

Instead, you:

* Call toolshed
* Preserve raw outputs
* Add interpretation layers

This keeps:

* Auditability
* Extensibility
* Replaceability

Exactly right.

---

## 4. The `calculate_statistical_assessments()` Aggregator Is the Right Abstraction

This function is 🔥:

```python
calculate_statistical_assessments(...)
```

Why it’s important:

* Single call for the orchestrator
* Clean boundary between “analysis” and “state”
* Future-proof for trends, forecasting, A/B tests

This is the **node-facing API** you want.

---

# High-Value Refinements (Strongly Recommended)

These are **small changes** that unlock long-term clarity and safety.

---

## 🔧 1. Normalize `assessment` vs `comprehensive_assessment`

Right now you sometimes return:

* `assessment`
* sometimes `comprehensive_assessment`
* sometimes both

That’s fine internally, but for orchestration clarity, I recommend **one rule**:

### Recommendation

Always return:

```python
{
  "raw_assessment": {...},      # toolshed output
  "executive_summary": str,     # single sentence
  "confidence_level": float
}
```

### Example Fix (Minimal Change)

Inside `assess_kpi_statistical_significance`:

```python
result["raw_assessment"] = assessment
result["executive_summary"] = assessment.get(
    "executive_summary",
    "Statistical assessment completed"
)
```

Why:

* Report generation becomes trivial
* LLM summaries have a clean input
* No branching logic downstream

This is a **big simplifier** later.

---

## 🔧 2. Explicitly Tag “Statistical Readiness”

Add one boolean field everywhere:

```python
"is_statistically_valid": bool
```

Example:

```python
result["is_statistically_valid"] = (
    historical_values is not None and len(historical_values) >= 2
)
```

Why this matters:

* Reports can say: *“Directional only”*
* Recommendations can downgrade confidence
* Alerts won’t fire on weak signals

Executives love clarity around *signal strength*.

---

## 🔧 3. Trend Analysis: Stub Is Fine — But Signal It Clearly

You did this:

```python
# For MVP, we'll skip trend analysis
```

That’s good — but I recommend returning an explicit placeholder:

```python
trends = {
    "status": "not_available",
    "reason": "Insufficient time-series data"
}
```

Why:

* Prevents downstream assumptions
* Makes future activation trivial
* Improves report honesty

---

## 🔧 4. KPI Naming Consistency (Future-Proofing)

Right now you assume:

```python
historical_data.get(kpi_name)
```

This is fine — **as long as KPI names never change**.

Future-proof option (optional, not urgent):

```python
def normalize_kpi_key(kpi_name: str) -> str:
    return kpi_name.lower().replace(" ", "_")
```

Use only if you later ingest external data.

---

# How This Should Be Wired Into the Orchestrator (Next Step)

You are now ready for a **new node**.

## ✅ `statistical_assessment_node`

### Inputs (from state)

* `operational_kpis`
* `effectiveness_kpis`
* `business_kpis`
* `total_cost_usd`
* `net_roi_usd`
* optional `historical_data`

### Node Skeleton

```python
def statistical_assessment_node(state, config):
    assessments = calculate_statistical_assessments(
        operational_kpis=state["operational_kpis"],
        effectiveness_kpis=state["effectiveness_kpis"],
        business_kpis=state["business_kpis"],
        total_cost_usd=state["total_cost_usd"],
        net_roi_usd=state["net_roi_usd"],
        historical_data=state.get("historical_data"),
        config=config
    )

    return {
        "statistical_assessments": assessments,
        "confidence_intervals": assessments["confidence_intervals"],
        "trends": assessments["trends"]
    }
```

Then place it **between KPI and ROI or just after ROI**, per your earlier plan.

---

# CEO Trust Test (This Passes)

If an executive asked:

> “How confident are we in these improvements?”

Your system can now answer:

* With p-values
* With confidence intervals
* With clear disclaimers
* Without hand-waving

That’s rare — and extremely valuable.

---

# Final Assessment

This utilities module is:

* ✅ Architecturally clean
* ✅ Statistically responsible
* ✅ Enterprise-safe
* ✅ Ready to wire into the orchestrator

You are officially past the “agent demo” phase.

You’re building **decision infrastructure**.



These refinements are worth implementing. They improve clarity, maintainability, and executive trust. Implementing them and fixing the test failure:

---

Implemented all of ChatGPT's refinements. Summary:

## Refinements implemented

### 1. Normalized assessment structure
- Always returns: `raw_assessment`, `executive_summary`, `confidence_level`
- Keeps `comprehensive_assessment` and `assessment` for backward compatibility
- Simplifies report generation and future LLM integration

### 2. Explicit statistical validity tagging
- Added `is_statistically_valid` boolean to all assessments
- Reports can clearly indicate "directional only" vs "statistically validated"
- Enables confidence-based recommendations and alerts

### 3. Explicit trend analysis status
- Returns clear status: `{"status": "not_available", "reason": "..."}`
- Prevents downstream assumptions
- Makes future activation straightforward

### 4. KPI name normalization (future-proofing)
- Added `normalize_kpi_key()` helper function
- Handles variations in KPI naming
- Supports external data ingestion

### 5. Fixed confidence_intervals issue
- Ensures `confidence_intervals` is always returned (even if empty)
- Fixed the test failure you saw
- Added explicit empty dict handling

## Benefits

1. Executive clarity: Reports can say "Directional only" vs "Statistically significant"
2. Cleaner code: Consistent structure makes report generation simpler
3. Future-proof: Ready for LLM summaries, external data, trend analysis
4. Better debugging: Clear status messages help diagnose issues

## Test it

The test should pass now. Run:
```bash
python3 test_proposal_document_orchestrator_statistical.py
```

These changes improve maintainability and executive trust without changing core functionality. The code is now more robust and ready for production use.

In [ ]:
"""Statistical Assessment Utilities for Proposal & Document Orchestrator

These utilities add statistical rigor to KPI and ROI analysis using toolshed.statistics.
Following CEO Trust patterns: statistical significance, confidence intervals, trend analysis.

Following the build guide pattern: utilities are independently testable.
"""

from typing import Dict, Any, List, Optional
from toolshed.statistics import (
    test_kpi_significance,
    test_roi_significance,
    test_trend_significance,
    calculate_kpi_confidence_interval,
    calculate_roi_confidence_interval,
    assess_kpi_with_significance,
    assess_roi_with_significance
)


def normalize_kpi_key(kpi_name: str) -> str:
    """
    Normalize KPI name for consistent lookup.

    Future-proofing: Ensures KPI names are normalized for historical data lookup,
    even if naming conventions change or external data is ingested.

    Args:
        kpi_name: KPI name (may have spaces, mixed case, etc.)

    Returns:
        Normalized key (lowercase, underscores)
    """
    return kpi_name.lower().replace(" ", "_").replace("-", "_")


def assess_kpi_statistical_significance(
    kpi_name: str,
    current_value: float,
    historical_values: Optional[List[float]] = None,
    target_value: Optional[float] = None,
    confidence_level: float = 0.95
) -> Dict[str, Any]:
    """
    Assess KPI with statistical significance testing.

    Args:
        kpi_name: Name of the KPI
        current_value: Current KPI value
        historical_values: Optional list of historical values
        target_value: Optional target value
        confidence_level: Confidence level (default 0.95)

    Returns:
        Dictionary with statistical assessment:
        {
            "kpi_name": str,
            "current_value": float,
            "statistical_test": {...},
            "confidence_interval": {...},
            "target_comparison": {...},
            "assessment": str
        }
    """
    # Determine statistical validity
    is_statistically_valid = historical_values is not None and len(historical_values) >= 2

    result = {
        "kpi_name": kpi_name,
        "current_value": current_value,
        "has_historical_data": is_statistically_valid,
        "is_statistically_valid": is_statistically_valid
    }

    # Statistical significance test (if historical data available)
    if historical_values and len(historical_values) >= 2:
        statistical_test = test_kpi_significance(
            current_value=current_value,
            historical_values=historical_values,
            target_value=target_value,
            confidence_level=confidence_level
        )
        result["statistical_test"] = statistical_test

        # Confidence interval
        try:
            ci = calculate_kpi_confidence_interval(
                kpi_value=current_value,
                sample_size=len(historical_values),
                confidence_level=confidence_level,
                is_proportion=(0 <= current_value <= 1)
            )
            result["confidence_interval"] = ci
        except Exception:
            result["confidence_interval"] = None
    else:
        result["statistical_test"] = {
            "error": "Insufficient historical data for statistical testing",
            "is_significant": None
        }
        result["confidence_interval"] = None

    # Comprehensive assessment (if historical data available)
    if is_statistically_valid:
        try:
            assessment = assess_kpi_with_significance(
                current_value=current_value,
                historical_values=historical_values,
                target_value=target_value,
                confidence_level=confidence_level
            )
            # Normalize output structure
            result["raw_assessment"] = assessment
            result["executive_summary"] = assessment.get("executive_summary", "Statistical assessment completed")
            result["confidence_level"] = confidence_level
            # Keep for backward compatibility
            result["comprehensive_assessment"] = assessment
            result["assessment"] = result["executive_summary"]
        except Exception as e:
            result["raw_assessment"] = None
            result["executive_summary"] = f"Assessment completed (limited: {str(e)})"
            result["confidence_level"] = confidence_level
            result["assessment"] = result["executive_summary"]
    else:
        result["raw_assessment"] = None
        result["executive_summary"] = "No historical data available for statistical assessment"
        result["confidence_level"] = confidence_level
        result["assessment"] = result["executive_summary"]

    return result


def assess_roi_statistical_significance(
    roi_estimate: float,
    cost: float,
    historical_roi: Optional[List[float]] = None,
    confidence_level: float = 0.95
) -> Dict[str, Any]:
    """
    Assess ROI with statistical significance testing.

    Args:
        roi_estimate: Current ROI estimate
        cost: Current cost
        historical_roi: Optional list of historical ROI values
        confidence_level: Confidence level (default 0.95)

    Returns:
        Dictionary with ROI statistical assessment:
        {
            "roi_estimate": float,
            "cost": float,
            "statistical_test": {...},
            "confidence_interval": {...},
            "assessment": str
        }
    """
    # Determine statistical validity
    is_statistically_valid = historical_roi is not None and len(historical_roi) >= 2

    result = {
        "roi_estimate": roi_estimate,
        "cost": cost,
        "roi_ratio": roi_estimate / cost if cost > 0 else float('inf'),
        "has_historical_data": is_statistically_valid,
        "is_statistically_valid": is_statistically_valid
    }

    # Statistical significance test (if historical data available)
    if historical_roi and len(historical_roi) >= 2:
        statistical_test = test_roi_significance(
            roi_estimate=roi_estimate,
            cost=cost,
            historical_roi=historical_roi,
            confidence_level=confidence_level
        )
        result["statistical_test"] = statistical_test

        # Confidence interval
        try:
            ci = calculate_roi_confidence_interval(
                roi_estimate=roi_estimate,
                cost=cost,
                historical_roi=historical_roi,
                confidence_level=confidence_level
            )
            result["confidence_interval"] = ci
        except Exception:
            result["confidence_interval"] = None
    else:
        result["statistical_test"] = {
            "error": "Insufficient historical data for statistical testing",
            "is_significantly_positive": None
        }
        result["confidence_interval"] = None

    # Comprehensive assessment (if historical data available)
    if is_statistically_valid:
        try:
            assessment = assess_roi_with_significance(
                roi_estimate=roi_estimate,
                cost=cost,
                historical_roi=historical_roi,
                confidence_level=confidence_level
            )
            # Normalize output structure
            result["raw_assessment"] = assessment
            result["executive_summary"] = assessment.get("executive_summary", "ROI statistical assessment completed")
            result["confidence_level"] = confidence_level
            # Keep for backward compatibility
            result["comprehensive_assessment"] = assessment
            result["assessment"] = result["executive_summary"]
        except Exception as e:
            result["raw_assessment"] = None
            result["executive_summary"] = f"Assessment completed (limited: {str(e)})"
            result["confidence_level"] = confidence_level
            result["assessment"] = result["executive_summary"]
    else:
        result["raw_assessment"] = None
        result["executive_summary"] = "No historical data available for statistical assessment"
        result["confidence_level"] = confidence_level
        result["assessment"] = result["executive_summary"]

    return result


def analyze_kpi_trends(
    kpi_name: str,
    values: List[float],
    confidence_level: float = 0.95
) -> Dict[str, Any]:
    """
    Analyze trends in KPI values over time.

    Args:
        kpi_name: Name of the KPI
        values: List of KPI values over time (chronologically ordered)
        confidence_level: Confidence level (default 0.95)

    Returns:
        Dictionary with trend analysis:
        {
            "kpi_name": str,
            "trend_analysis": {...},
            "trend_direction": str,
            "is_significant": bool
        }
    """
    if len(values) < 3:
        return {
            "kpi_name": kpi_name,
            "trend_analysis": {
                "error": "Insufficient data for trend analysis (need at least 3 observations)"
            },
            "trend_direction": "insufficient_data",
            "is_significant": False
        }

    # Test trend significance
    trend_analysis = test_trend_significance(
        values=values,
        confidence_level=confidence_level
    )

    return {
        "kpi_name": kpi_name,
        "trend_analysis": trend_analysis,
        "trend_direction": trend_analysis.get("trend_direction", "unknown"),
        "is_significant": trend_analysis.get("is_significant", False)
    }


def assess_all_kpis_statistically(
    operational_kpis: Dict[str, Any],
    effectiveness_kpis: Dict[str, Any],
    business_kpis: Dict[str, Any],
    historical_data: Optional[Dict[str, List[float]]] = None,
    config: Any = None
) -> Dict[str, Any]:
    """
    Assess all KPIs with statistical significance testing.

    Args:
        operational_kpis: Operational KPI metrics
        effectiveness_kpis: Effectiveness KPI metrics
        business_kpis: Business KPI metrics
        historical_data: Optional dictionary mapping KPI names to historical values
        config: Agent configuration (for confidence_level and targets)

    Returns:
        Dictionary with statistical assessments for all KPIs:
        {
            "operational_kpis": {...},
            "effectiveness_kpis": {...},
            "business_kpis": {...},
            "summary": {...}
        }
    """
    confidence_level = getattr(config, "confidence_level", 0.95) if config else 0.95

    assessments = {
        "operational_kpis": {},
        "effectiveness_kpis": {},
        "business_kpis": {},
        "summary": {
            "total_kpis_assessed": 0,
            "kpis_with_historical_data": 0,
            "statistically_significant_improvements": 0,
            "statistically_significant_declines": 0
        }
    }

    # Assess operational KPIs
    for kpi_name, kpi_value in operational_kpis.items():
        # Normalize key for historical data lookup (future-proofing)
        normalized_key = normalize_kpi_key(kpi_name)
        historical_values = (
            historical_data.get(kpi_name) or
            historical_data.get(normalized_key)
            if historical_data else None
        )
        target_value = getattr(config, f"target_{kpi_name}", None) if config else None

        assessment = assess_kpi_statistical_significance(
            kpi_name=kpi_name,
            current_value=kpi_value,
            historical_values=historical_values,
            target_value=target_value,
            confidence_level=confidence_level
        )

        assessments["operational_kpis"][kpi_name] = assessment
        assessments["summary"]["total_kpis_assessed"] += 1

        if assessment.get("has_historical_data"):
            assessments["summary"]["kpis_with_historical_data"] += 1
            if assessment.get("statistical_test", {}).get("is_significant"):
                if assessment["statistical_test"].get("difference", 0) > 0:
                    assessments["summary"]["statistically_significant_improvements"] += 1
                else:
                    assessments["summary"]["statistically_significant_declines"] += 1

    # Assess effectiveness KPIs
    for kpi_name, kpi_value in effectiveness_kpis.items():
        # Normalize key for historical data lookup (future-proofing)
        normalized_key = normalize_kpi_key(kpi_name)
        historical_values = (
            historical_data.get(kpi_name) or
            historical_data.get(normalized_key)
            if historical_data else None
        )
        target_value = getattr(config, f"target_{kpi_name}", None) if config else None

        assessment = assess_kpi_statistical_significance(
            kpi_name=kpi_name,
            current_value=kpi_value,
            historical_values=historical_values,
            target_value=target_value,
            confidence_level=confidence_level
        )

        assessments["effectiveness_kpis"][kpi_name] = assessment
        assessments["summary"]["total_kpis_assessed"] += 1

        if assessment.get("has_historical_data"):
            assessments["summary"]["kpis_with_historical_data"] += 1
            if assessment.get("statistical_test", {}).get("is_significant"):
                if assessment["statistical_test"].get("difference", 0) > 0:
                    assessments["summary"]["statistically_significant_improvements"] += 1
                else:
                    assessments["summary"]["statistically_significant_declines"] += 1

    # Assess business KPIs
    for kpi_name, kpi_value in business_kpis.items():
        # Normalize key for historical data lookup (future-proofing)
        normalized_key = normalize_kpi_key(kpi_name)
        historical_values = (
            historical_data.get(kpi_name) or
            historical_data.get(normalized_key)
            if historical_data else None
        )
        target_value = getattr(config, f"target_{kpi_name}", None) if config else None

        assessment = assess_kpi_statistical_significance(
            kpi_name=kpi_name,
            current_value=kpi_value,
            historical_values=historical_values,
            target_value=target_value,
            confidence_level=confidence_level
        )

        assessments["business_kpis"][kpi_name] = assessment
        assessments["summary"]["total_kpis_assessed"] += 1

        if assessment.get("has_historical_data"):
            assessments["summary"]["kpis_with_historical_data"] += 1
            if assessment.get("statistical_test", {}).get("is_significant"):
                if assessment["statistical_test"].get("difference", 0) > 0:
                    assessments["summary"]["statistically_significant_improvements"] += 1
                else:
                    assessments["summary"]["statistically_significant_declines"] += 1

    return assessments


def calculate_statistical_assessments(
    operational_kpis: Dict[str, Any],
    effectiveness_kpis: Dict[str, Any],
    business_kpis: Dict[str, Any],
    total_cost_usd: float,
    net_roi_usd: float,
    historical_data: Optional[Dict[str, Any]] = None,
    config: Any = None
) -> Dict[str, Any]:
    """
    Calculate complete statistical assessments for KPIs and ROI.

    Args:
        operational_kpis: Operational KPI metrics
        effectiveness_kpis: Effectiveness KPI metrics
        business_kpis: Business KPI metrics
        total_cost_usd: Total cost
        net_roi_usd: Net ROI
        historical_data: Optional historical data dictionary
        config: Agent configuration

    Returns:
        Complete statistical assessments:
        {
            "kpi_assessments": {...},
            "roi_assessment": {...},
            "confidence_intervals": {...},
            "trends": {...}
        }
    """
    confidence_level = getattr(config, "confidence_level", 0.95) if config else 0.95

    # Extract historical KPI data
    historical_kpi_data = None
    historical_roi_data = None

    if historical_data:
        historical_kpi_data = historical_data.get("kpis", {})
        historical_roi_data = historical_data.get("roi", [])

    # Assess all KPIs
    kpi_assessments = assess_all_kpis_statistically(
        operational_kpis=operational_kpis,
        effectiveness_kpis=effectiveness_kpis,
        business_kpis=business_kpis,
        historical_data=historical_kpi_data,
        config=config
    )

    # Assess ROI
    roi_assessment = assess_roi_statistical_significance(
        roi_estimate=net_roi_usd,
        cost=total_cost_usd,
        historical_roi=historical_roi_data,
        confidence_level=confidence_level
    )

    # Extract confidence intervals
    confidence_intervals = {}

    # KPI confidence intervals
    for category in ["operational_kpis", "effectiveness_kpis", "business_kpis"]:
        for kpi_name, assessment in kpi_assessments[category].items():
            ci = assessment.get("confidence_interval")
            if ci:
                confidence_intervals[kpi_name] = ci

    # ROI confidence interval
    if roi_assessment.get("confidence_interval"):
        confidence_intervals["roi"] = roi_assessment["confidence_interval"]

    # Trend analysis (if we have time series data)
    # For MVP, we'll skip trend analysis as it requires time series data
    # This can be added later when historical data is available
    trends = {
        "status": "not_available",
        "reason": "Insufficient time-series data for trend analysis",
        "note": "Trend analysis requires chronological time-series data across multiple periods"
    }

    return {
        "kpi_assessments": kpi_assessments,
        "roi_assessment": roi_assessment,
        "confidence_intervals": confidence_intervals,
        "trends": trends
    }
